For now, starter code that compares lines from the two files and prints out the differences.

In [1]:
import json, csv
import pandas as pd
from constants import *
import numpy as np
from scipy.stats import poisson

def poisson_over_likelyhood(mean, target):
    return 1 - poisson.cdf(target, mean)



The following code will web scrape to generate the player data, feel free to ingore

In [2]:
import fbref_scrape
# import pp_scrape

https://fbref.com/en/comps/676/stats/UEFA-Euro-Stats
200.  We're in
https://fbref.com/en/squads/1862c019/England-Men-Stats
200.  We're in
https://fbref.com/en/players/4806ec67/matchlogs/nat_tm/Jordan-Pickford-Match-Logs
200.  We're in
Date
2017-11-10
2018-03-23
2018-06-02
2018-06-18
2018-06-24
2018-06-28
2018-07-03
2018-07-07
2018-07-11
2018-07-14
2018-09-08
2018-10-12
2018-10-15
2018-11-15
2018-11-18
2019-03-22
2019-03-25
2019-06-06
2019-06-09
2019-09-07
2019-09-10
2019-10-11
2019-10-14
2019-11-14
2020-09-05
2020-09-08
2020-10-11
2020-10-14
2020-11-15
2020-11-18
2021-06-02
2021-06-13
2021-06-18
2021-06-22
2021-06-29
2021-07-03
2021-07-07
2021-07-11
2021-09-02
2021-09-08
2021-10-12
2021-11-12
2022-03-26
2022-06-04
2022-06-07
2022-11-21
2022-11-25
2022-11-29
2022-12-04
2022-12-10
2023-03-23
2023-03-26
2023-06-16
2023-06-19
2023-09-09
2023-10-17
2023-11-17
2023-11-20
2024-03-23
2024-03-26
2024-06-03
2024-06-16
2024-06-20
2024-06-25
2024-06-30
2024-07-06

https://fbref.com/en/players/1c70

In [3]:
# euros = fbref_scrape.SoccerLeague("https://fbref.com/en/comps/676/stats/UEFA-Euro-Stats", [6, 7, 12, 20])


In [4]:
players = pd.read_csv("Premier_League_Player_Stats.csv")
PP_lines = pd.read_csv("pp_lines.csv")

merged = pd.merge(players, PP_lines, on="name")
merged

,name,team,games,minutes,gk_saves,clearances,tackles,assisted_shots,fouled,shots,...,Goalie Saves,Tackles,Touches,Shots,Attempted Dribbles,Fouls,Shots On Target,Shots Assisted,Clearances,Fouls Drawn


In [5]:
merged = merged.loc[merged['minutes'] > 200].loc[merged['games'] > 5]

merged.loc[merged['Goalie Saves'] > 0].head()

,name,team,games,minutes,gk_saves,clearances,tackles,assisted_shots,fouled,shots,...,Goalie Saves,Tackles,Touches,Shots,Attempted Dribbles,Fouls,Shots On Target,Shots Assisted,Clearances,Fouls Drawn


We need a new dataframe such that each row repesents a singular betting line that has a per-90 line, a per-game line, and a prizepicks line

In [6]:
results = []

for index, row in merged.iterrows():
    # Access the values of each column in the current row
    name = row['name']

    for pp_stat_type in pp_to_fbref_stats.keys():
        # failsafe 
        if pp_stat_type not in row:
            continue

        if row[pp_stat_type] <= 0:
            continue

        # Create a dictionary to store the result
        result_dict = {
            'name': name,
            'stat': pp_stat_type,
            'line': row[pp_stat_type]
        }

        # Get the FBref stat type that corresponds to the current PP stat type
        fbref_stat_type = pp_to_fbref_stats[pp_stat_type]

        # Get the value of the FBref stat type for the current player
        fbref_stat_value = row[fbref_stat_type]

        # Add the FBref stat value to the result dictionary
        result_dict['per90'] = fbref_stat_value * 90 / row['minutes']
        result_dict['perGame'] = fbref_stat_value / row['games']


        # Append the dictionary to the results list
        results.append(result_dict)

# Convert the results list into a dataframe
df = pd.DataFrame(results).dropna()

df


""


Now that we have that sorted, we can analyze the chart for interesting discrepencies in data

In [7]:
df = df.assign(quo = df['line'] / df['per90']).assign(diff = df['line'] - df['per90']).assign(over_likelyhood = poisson_over_likelyhood(df['per90'], df['line']))

df = df.sort_values(by='over_likelyhood', ascending=False)

# show the full thing
pd.set_option('display.max_rows', None)
df

KeyError: 'line'